In [1]:
import numpy as np
np.random.seed(2671)  # for reproducibility

import copy
import json
import cv2
from scipy.misc import imread

from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from keras.utils.layer_utils import print_summary
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import sys


Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5103)
/Library/Python/2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
catmapfilename='breed_map.list'
catmapfile = open(catmapfilename,'r')
categoryMap = dict()
for line in catmapfile.readlines():
    ls = line.split()
    category = ls[0]
    breed = ls[1]
    categoryMap[category] = breed

categoryIndex = dict(zip(categoryMap.keys(),range(len(categoryMap))))
nb_classes = len(categoryMap)

In [3]:
input_photos = []
breedlist = []
posfilename = "positives_mod10.txt"
inputdirname = "CroppedDogs/"
posfile = open(posfilename,'r')
for line in posfile.readlines():
    infile = line.strip()
    ls0 = infile.split("/")
    ls1 = ls0[1].split("_")
    category = ls1[0]
    img = cv2.resize(cv2.imread(infile), (299, 299), cv2.INTER_LINEAR)
    input_photos.append(img)
    breedlist.append(categoryIndex[category])

In [4]:
print len(input_photos),len(breedlist)
batch_size = 10
nb_epoch = 5

train = np.array(input_photos)
labels = np.array(breedlist)

print train.shape

nToUse = len(train)
testFraction = 1.0/3.0
nTest = int(testFraction*nToUse)

X_test = train[:nTest]
y_test = labels[:nTest]

data_augmentation = False

# convert class vectors to binary class matrices
Y_test = np_utils.to_categorical(y_test, nb_classes)

# input image dimensions
img_rows, img_cols = X_test.shape[1], X_test.shape[2]

if K.image_dim_ordering() == 'th':
    X_test = np.swapaxes(X_test,1,3)
    X_test = np.swapaxes(X_test,2,3)
    input_shape = (X_test.shape[1], img_rows, img_cols)
else:
	input_shape = (img_rows, img_cols, X_test.shape[3])

print input_shape

X_test = X_test.astype('float32')
X_test /= 255

2212 2212
(2212, 299, 299, 3)
(3, 299, 299)


In [5]:
model = load_model("dog_cnn_simple.keras")

In [6]:
#score = model.evaluate(X_test, Y_test, verbose=0)
Y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(Y_pred)
testID = 1100
similarity100 = similarities[testID,:]

In [16]:
import operator

sorted_x = sorted(zip(range(len(similarity100)),similarity100), key=operator.itemgetter(1),reverse=True)
print sorted_x[:5]
fig = plt.figure(figsize=(8,6))
from matplotlib import gridspec
gs = gridspec.GridSpec(5,5)
axtop = plt.subplot(gs[0:1,:])
axtop.imshow(cv2.cvtColor(input_photos[testID], cv2.COLOR_BGR2RGB))
axb1 = plt.subplot(gs[2:,0])
axb1.imshow(cv2.cvtColor(input_photos[sorted_x[0][0]], cv2.COLOR_BGR2RGB))
axb2 = plt.subplot(gs[2:,1])
axb2.imshow(cv2.cvtColor(input_photos[sorted_x[1][0]], cv2.COLOR_BGR2RGB))
axb3 = plt.subplot(gs[2:,2])
axb3.imshow(cv2.cvtColor(input_photos[sorted_x[2][0]], cv2.COLOR_BGR2RGB))
axb4 = plt.subplot(gs[2:,3])
axb4.imshow(cv2.cvtColor(input_photos[sorted_x[3][0]], cv2.COLOR_BGR2RGB))
axb5 = plt.subplot(gs[2:,4])
axb5.imshow(cv2.cvtColor(input_photos[sorted_x[4][0]], cv2.COLOR_BGR2RGB))
fig.savefig("testOutput_"+str(testID)+".jpg")

[(357, 1.0), (495, 1.0), (496, 1.0), (497, 1.0), (498, 1.0)]
